# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from datetime import datetime
from sql_queries import *


In [17]:
# %load sql_queries.py
# DROP TABLES

songplay_table_drop = "DROP TABLE songplays"
user_table_drop = "DROP TABLE users"
song_table_drop = "DROP TABLE songs"
artist_table_drop = "DROP TABLE artists"
time_table_drop = "DROP TABLE time"

# CREATE TABLES

songplay_table_create = (""" CREATE TABLE IF NOT EXISTS songplays (songplay_id int, start_time timestamp, user_id int, level varchar,\
                                song_id varchar, artist_id varchar, session_id int, location varchar, user_agent varchar)
                        """)

user_table_create = (""" CREATE TABLE IF NOT EXISTS users (user_id int, first_name varchar, last_name varchar, gender varchar, level varchar)
""")

song_table_create = (""" CREATE TABLE IF NOT EXISTS songs (song_id varchar, title varchar, artist_id varchar, year int, duration float) 
""")

artist_table_create = (""" CREATE TABLE IF NOT EXISTS artists (artist_id varchar, name varchar, location varchar, latitude float, longitude float)
""")

time_table_create = (""" CREATE TABLE IF NOT EXISTS time (start_time timestamp, hour int, day int, week int, month int, year int)
""")

# INSERT RECORDS

songplay_table_insert = (""" INSERT INTO songplays (songplay_id , start_time , user_id , level, song_id, artist_id, session_id , location , user_agent )\
                                VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)
                                 ON CONFLICT DO NOTHING
""")

user_table_insert = (""" INSERT INTO users (user_id, first_name, last_name, gender, level) VALUES(%s, %s, %s, %s, %s)
""")

song_table_insert = (""" INSERT INTO songs (song_id, title, artist_id, year, duration) VALUES(%s, %s, %s, %s, %s )
""")

artist_table_insert = (""" INSERT INTO artists (artist_id, name, location, latitude, longitude ) VALUES(%s, %s, %s, %s, %s)
""")


time_table_insert = (""" INSERT INTO time (start_time, hour, day, week, month, year) VALUES (%s, %s, %s, %s, %s, %s)
""")

# FIND SONGS

song_select = (""" SELECT * from songs s join artists a on a.artist_id = s.artist_id WHERE s.title = %s and a.name = %s 
""")

# QUERY LISTS

create_table_queries = [songplay_table_create, user_table_create, song_table_create, artist_table_create, time_table_create]
drop_table_queries = [songplay_table_drop, user_table_drop, song_table_drop, artist_table_drop, time_table_drop]

In [3]:
 # connect to default database
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

# create sparkify database with UTF8 encoding
cur.execute("DROP DATABASE IF EXISTS sparkifydb")
cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")

# close connection to default database
conn.close()    

# connect to sparkify database
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
    

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [12]:
#
song_files = get_files("data/song_data")


In [13]:
filepath = song_files[1]

In [14]:
df = pd.read_json(filepath, lines=True) 
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR558FS1187FB45658,NaN,,NaN,40 Grit,75.67628,1,SOGDBUF12A8C140FAA,Intro,2003


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [15]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]
song_data

['SOGDBUF12A8C140FAA', 'Intro', 'AR558FS1187FB45658', 2003, 75.67628]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [16]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [17]:
artist_data = df[[ 'artist_id', 'artist_name', 'artist_location','artist_latitude', 'artist_longitude']].values.tolist()[0]
artist_data

['AR558FS1187FB45658', '40 Grit', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [18]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [7]:
log_files = get_files('data/log_data')

In [8]:
filepath = log_files[0]

In [9]:
df =  pd.read_json(filepath, lines=True) 
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [10]:
df = df.loc[df['page'] == 'NextSong']
df.head(20)
df.count()

artist           319
auth             319
firstName        319
gender           319
itemInSession    319
lastName         319
length           319
level            319
location         319
method           319
page             319
registration     319
sessionId        319
song             319
status           319
ts               319
userAgent        319
userId           319
dtype: int64

In [11]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()
t.count()

319

In [12]:
time_df = pd.DataFrame(data = [])
time_df['start_time'] = t
time_df['hour'] = t.dt.hour
time_df['day'] = t.dt.day
time_df['week'] = t.dt.week
time_df['month'] = t.dt.month
time_df['year'] = t.dt.year
time_df.head()

,start_time,hour,day,week,month,year
0,2018-11-29 00:00:57.796,0,29,48,11,2018
1,2018-11-29 00:01:30.796,0,29,48,11,2018
2,2018-11-29 00:04:01.796,0,29,48,11,2018
3,2018-11-29 00:04:55.796,0,29,48,11,2018
4,2018-11-29 00:07:13.796,0,29,48,11,2018


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [13]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [14]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [15]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    #cur.execute(song_select, (row.song, row.artist, row.length))
    print((row.song.replace(" \" ", "")
           , row.artist))
    cur.execute(song_select, (row.song, row.artist))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, pd.to_datetime(df['ts'][index], unit='ms'), row.userId, row.level, songid, artistid, row.itemInSession,\
                    row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    

("Ain't No Sunshine", 'Sydney Youngblood')
('My Advice 2 You (Explicit)', 'Gang Starr')
('My First Kiss (Feat. Ke$ha) [Album Version]', '3OH!3')
('The Girl and The Robot', 'RÃ\x83Â¶yksopp')
('Too Shy', 'Kajagoogoo')
('Potential', 'The Jimmy Castor Bunch')
('The Great Satan (What Would Satan Do Mix)', 'Ministry')
('The Screamer', 'Bouncing Souls')
('Knife Going In (Album Version)', 'Tegan And Sara')
('The Greatest View (Album Version)', 'Silverchair')
('T.N.T. For The Brain (112 Bpm) (Radio Edit)', 'Enigma')
('Mon MÃ\x83Â©tro', 'Anis')
('Fly Like An Eagle', 'Seal')
('Restless', 'UNKLE Feat. Josh Homme')
("You're The One", 'Dwight Yoakam')
('Blue', 'Elastica')
('Channel 10', 'Capone-N-Noreaga')
('The Oak Tree (Album Version)', 'Morris Day')
('What If I Do?', 'Foo Fighters')
("You're Not Alone", 'Olive')
("En Glad Calypso Om VÃ\x83Â¥ren (Live '62)", 'Olle Adolphson')
("Down_ Down_ Down To Mehphisto's Cafe (Album Version)", 'Streetlight Manifesto')
('Tu Eres Ajena', 'Frank Reyes')
('Estadi

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [33]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.